# Custom commands in the VR client

## Setup the server

We will follow a similar setup to our [getting started](getting_started.ipynb) tutorial notebook, using a pre-prepared methane & nanotube input file to run an iMD simulation using OpenMM as the physics engine. Let's start by importing the classes necessary to run the simulation.

In [ ]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation

Next, let's create an `OpenMMSimulation` object using our pre-prepared input file:

In [ ]:
# Create the simulation object
nanotube_simulation = OpenMMSimulation.from_xml_path("openmm_files/nanotube.xml")

Now we can set everything up to run the simulation, passing our `OpenMMSimulation` to an `OmniRunner` which will serve the simulation for iMD.

In [ ]:
# Start the runner/server
imd_runner = OmniRunner.with_basic_server(nanotube_simulation, name='vr-commands-server')

# Load the first simulation
imd_runner.load(0)

## Defining a command

We'll define a command that, when triggered, saves the current state of the current simulation and adds to the list of simulations in simulation selection: 

In [ ]:
def clone_omni_omm(omni_omm):
    """
    Create an independent copy of a given NanoVer OpenMMSimulation.
    """
    from nanover.openmm import serializer

    # get the underlying openmm simulation object
    omm = omni_omm.simulation

    # clone it by serializing and then deserializing
    data = serializer.serialize_simulation(omm)
    omm_clone = serializer.deserialize_simulation(data)

    # wrap the cloned openmm simulation in OpenMMSimulation
    omni_omm_clone = OpenMMSimulation.from_simulation(omm_clone)
    
    # give it different name
    omni_omm_clone.name = omni_omm.name + "'"
    
    return omni_omm_clone


def add_checkpoint_simulation():
    imd_runner.add_simulation(clone_omni_omm(imd_runner.simulation))


# add the command to the server,
# commands with names beginning with "user/" are displayed in the VR client
imd_runner.app_server.register_command("user/checkpoint", add_checkpoint_simulation)

## Using the command

The command is now available in the VR client. You can try it yourself in [Nanover iMD-VR](https://irl2.github.io/nanover-docs/installation.html#installing-the-imd-vr-client). Connect to the server and use the "User Commands" menu to trigger the command; it will add a new listing to the simulations selection menu, just like in this video:

<video controls src="images/imd-vr-checkpoint-command-small.mp4" alt="video demonstrating the command in vr" />

## Cleaning up

In [ ]:
# Close the runner
imd_runner.close()